In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
from tqdm.notebook import trange, tqdm
from collections import Counter

In [ ]:
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
train = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")

In [ ]:
def get_image_file_path_test(image_id):
    return f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg'

test['file_path'] = test['Id'].apply(get_image_file_path_test)

In [ ]:
test.head()

In [ ]:

def get_image_file_path_train(image_id):
    return f'/kaggle/input/petfinder-pawpularity-score/train/{image_id}.jpg'

train['file_path'] = train['Id'].apply(get_image_file_path_train)


In [ ]:
train.head()

In [ ]:
kaggle_train_images = []
i=0
for i,img_path in enumerate(tqdm(train['file_path'])):
  if img_path.split("/")[-1] in os.listdir("/kaggle/input/petfinder-pawpularity-score/train/"):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(128,128))
    kaggle_train_images.append(img)
  else:
    train = train.drop(i)
kaggle_train_images=np.array(kaggle_train_images)

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
fig = plt.figure()
sns.histplot(data=train, x='Pawpularity', bins=100,color ='green')
plt.axvline(train['Pawpularity'].mean(), c='red', ls='-', lw=6, label='Mean Pawpularity')
plt.axvline(train['Pawpularity'].median(),c='blue',ls='-',lw=6, label='Median Pawpularity')
plt.legend()
plt.show()

In [ ]:
kaggle_pawpularity_train = train["Pawpularity"]
kaggle_pawpularity_train = kaggle_pawpularity_train/100
kaggle_pawpularity_train = kaggle_pawpularity_train.values

In [ ]:
kaggle_train_images = np.load("./kaggle_train_images.npy")

In [ ]:
np.random.seed(0)
labels_train = train[["Subject Focus","Eyes","Face","Near","Action","Accessory","Group","Collage","Human","Occlusion","Info","Blur"]].values
def unison_shuffled_copies(a, b):
    assert a.shape[0] == b.shape[0]
    p = np.random.permutation(len(a))
    return p
p = unison_shuffled_copies(kaggle_train_images,kaggle_pawpularity_train)
all_images = kaggle_train_images[p]
answers = kaggle_pawpularity_train[p]
labels_train = labels_train[p]
print(answers[:3])
print(answers[:3])
super_test_x = all_images[9000:]
all_images = all_images[:9000]

super_test_y=answers[9000:]
answers=answers[:9000]

all_images = all_images / 255.0
super_test_x = super_test_x / 255.0

In [ ]:
answers

In [ ]:
np.save("./kaggle_train_images.npy",kaggle_train_images)
np.save("./kaggle_pawpularity_train.npy",kaggle_pawpularity_train)

In [ ]:
import tensorflow as tf
tf.random.set_seed(101)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,Activation,BatchNormalization,MaxPooling2D
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def Conv2D_NET(input_shape):
    init = tf.keras.initializers.GlorotUniform()
    reg = l2(0.001)
    chanDim = -1
    model = Sequential()
    
    model.add(Conv2D(16, (7, 7), strides=(2, 2), padding="valid",
        kernel_initializer=init, kernel_regularizer=reg,
        input_shape=input_shape))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))
              
              
    model.add(Conv2D(32, (3, 3), padding="same",
        kernel_initializer=init, kernel_regularizer=reg))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",
        kernel_initializer=init, kernel_regularizer=reg))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding="same",
        kernel_initializer=init, kernel_regularizer=reg))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same",
        kernel_initializer=init, kernel_regularizer=reg))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), padding="same",
        kernel_initializer=init, kernel_regularizer=reg))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    
    
    
    model.add(Flatten())
    
    model.add(Dense(512, kernel_initializer=init))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    # Compile
    model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])
    return model

In [ ]:
model = Conv2D_NET((128,128,3))

In [ ]:
model.summary()

In [ ]:
EarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=20)
checkpoint_path_quality = f"/kaggle/working/paws_c2d_v4.h5"

checkpoint = ModelCheckpoint(checkpoint_path_quality, 
                             monitor='val_rmse', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_rmse',
                                            patience=1,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=1e-15)


In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
model.fit(x=aug.flow(all_images, answers, batch_size=64),
	validation_data=aug.flow(super_test_x, super_test_y, batch_size=128), batch_size = 64,
	epochs=100,callbacks = [checkpoint,learning_rate_reduction,EarlyStopper])



In [ ]:
kaggle_test_images = []
i=0
for i,img_path in enumerate(test['file_path']):
  if img_path.split("/")[-1] in os.listdir("/kaggle/input/petfinder-pawpularity-score/test/"):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(128,128))
    kaggle_test_images.append(img)
  else:
    test = test.drop(i)
kaggle_test_images = np.array(kaggle_test_images)

In [ ]:
del all_images
del answers
del kaggle_train_images
del kaggle_pawpularity_train
del aug

In [ ]:
loaded_model = load_model("/kaggle/working/paws_c2d_v4.h5")
norm_test = np.copy(kaggle_test_images)
norm_test = norm_test / 255
test_pred = loaded_model.predict(norm_test,batch_size = 1)
test_pred = (test_pred*100)
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['Pawpularity']=test_pred
sub.to_csv('submission.csv',index=False)
sub.head(2)

In [ ]:
labels_train = train[["Subject Focus","Eyes","Face","Near","Action","Accessory","Group","Collage","Human","Occlusion","Info","Blur"]].values

In [ ]:
dense_model = Sequential()
dense_model.add(Dense(512,input_shape=(12,)))
dense_model.add(Dense(256,activation="relu"))
dense_model.add(Dense(1))
dense_model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [ ]:
EarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='rmse', patience=20)
checkpoint_path_quality = f"/kaggle/working/paws_dense.h5"

checkpoint = ModelCheckpoint(checkpoint_path_quality, 
                             monitor='rmse', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')
learning_rate_reduction = ReduceLROnPlateau(monitor='rmse',
                                            patience=1,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=1e-15)

In [ ]:
dense_model.fit(labels_train,kaggle_pawpularity_train, batch_size = 64,
	epochs=100,callbacks = [checkpoint,learning_rate_reduction,EarlyStopper])

In [ ]:
dense_model = load_model("/kaggle/working/paws_dense.h5")

In [ ]:
dense_pred = dense_model.predict(labels_train)

In [ ]:
train_pred = loaded_model.predict(all_images,batch_size =1 )

In [ ]:
dense_pred = dense_pred[:9000]

In [ ]:
train_pred.shape

In [ ]:
ens_vec = []
for c , d in zip(train_pred, dense_pred):
    ens_vec.append(np.array([c,d]))
ens_vec = np.array(ens_vec)

In [ ]:
ens_vec = ens_vec.reshape(9000,2)

In [ ]:

out_dense = Sequential()
out_dense.add(Dense(512,input_shape=(2,)))
out_dense.add(Dense(256,activation="relu"))
out_dense.add(Dense(1))
out_dense.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [ ]:
EarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='rmse', patience=20)
checkpoint_path_quality = f"/kaggle/working/paws_dense.h5"

checkpoint = ModelCheckpoint(checkpoint_path_quality, 
                             monitor='rmse', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')
learning_rate_reduction = ReduceLROnPlateau(monitor='rmse',
                                            patience=1,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=1e-15)

In [ ]:
out_dense.fit(ens_vec,answers, batch_size = 64,
	epochs=100,callbacks = [checkpoint,learning_rate_reduction,EarlyStopper])

In [ ]:
out_dense = load_model("/kaggle/working/paws_dense.h5")
norm_test = np.copy(kaggle_test_images)
norm_test = norm_test / 255
test_pred = out_dense.predict(norm_test,batch_size = 1)
test_pred = (test_pred*100)
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['Pawpularity']=test_pred
sub.to_csv('submission.csv',index=False)
sub.head(2)